VAE viewer notebook for JetBot
===

This notebook can visualize vae. This repository using JetBot real camera.

In [ ]:
import sys
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import torch
sys.path.append('../../vae')
from jetbot import Camera, bgr8_to_jpeg
from vae import VAE

In [ ]:
IMAGE_CHANNELS = 3
VARIANTS_SIZE = 32
MODEL_PATH = '../../vae.torch'
device = torch.device('cuda')


## Load trained VAE model.
Loading trained VAE model on GPU memory. 

In [ ]:
vae = VAE(image_channels=IMAGE_CHANNELS, z_dim=VARIANTS_SIZE)
vae.load_state_dict(torch.load(MODEL_PATH, map_location=torch.device(device)))
vae.to(device).eval()

## Create camera 


In [ ]:
camera = Camera.instance(width=320, height=240)

## Define preprocess and postprocess

In [ ]:
def preprocess(image):
    observe = PIL.Image.fromarray(image)
    observe = observe.resize((160,120))
    croped = observe.crop((0, 40, 160, 120))
    tensor = transforms.ToTensor()(croped)
    return tensor
    

def rgb8_to_jpeg(bytes):
    return bytes(cv2.imencode('.jpg', bytes)[1])

In [ ]:
image = widgets.Image(format='jpeg', width=320, height=240)
result = widgets.Image(format='jpeg', width=160, height=80)
camera_link = traitlets.dlink((camera,'value'), (image,'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image,result]))

## Start main process

In [ ]:
def vae_process(change):
    image = change['new']
    image = preprocess(image)
    z, _ ,_ = vae.encode(image.to(device))
    reconst = vae.decode(z)
    reconst = reconst.detach().cpu()[0].numpy()
    reconst = np.transpose(np.uint8(reconst*255),[1,2,0])
    result.value = rgb8_to_jpeg(reconst) 

vae_process({'new': camera.value})
camera.observe(vae_process, names='value')

## Clean up process

In [ ]:
camera.unobserve(vae_process, names='value')
camera_link.unlink()
